In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
params = np.load("cos_uniform_cut_data/cos_uniform_cut_0.npy")[:, 0:3]
lensed = np.load("cos_uniform_cut_data/cos_uniform_cut_0_output.npy")[:, 2:5000, 0]
unlensed = np.load("cos_uniform_cut_data/cos_uniform_cut_0_unlensed_output.npy")[:, :, 0]

real_lensing = lensed/unlensed

FileNotFoundError: [Errno 2] No such file or directory: 'cos_uniform_cut_data/cos_uniform_cut_0.npy'

In [243]:
print(real_lensing.shape)
print(params.shape)

# params_0 = ombh2
# params_1 = omch2


(2000, 4998)
(2000, 3)


In [1]:
def lensing_tail_fit(ell, ombh2, omch2):
    
    """
    Cl_lensed(ell) = lensing_tail_fit(ell; ombh2, omch2) * Cl_unlensed(ell)

    The function lensing_tail_fit is given by: 
    
        lensing_tail_fit(ell; ombh2, omch2) = 1 + [beta_1*(ell/beta_2)^alpha(ombh2, omch2) - 1] * [(1 + np.exp(-(ell - beta_3*2000)/100))^-1],
        
    where beta_1, beta_2, beta_3, and alpha(ombh2, omch2) were learned with PySR as:
    
        beta_1 = 0.83
        beta_2 = 1400/0.4531
        beta_3 = 1.62

        alpha(ombh2, omch2) = (ombh2^-0.877) * (ombh2 + omch2) - 3.342*(ombh2 + omch2) - 1.118

    Inputs:
        ell:   Single value of ell or numpy array of ell values to be evaluated.
        ombh2: Baryon Density Parameter for Cosmology of interest.
        omch2: Dark Matter Density Parameter for Cosmology of interest.

    Outputs: 
        L: Lensing tail for specified ell(s). Returns same type of object as input ell (single value or numpy array).
    
    """
    
    beta_1 = 0.83
    beta_2 = 3218
    beta_3 = 3240
    
    # Sigmoid
    sigm = (1 + np.exp(-(ell-beta_3)/100))**(-1)

    # Exponent
    alpha = (ombh2**-0.877) * (ombh2 + omch2) - 3.342*(ombh2 + omch2) - 1.118

    # Polynomial
    poly = ((beta_1*(ell/beta_2)**alpha - 1)*sigm) + 1

    L = poly*sigm + 1
    
    return L, poly, sigm

In [2]:
ells = np.array([i for i in range(2, 5000)])

counter = 0

for i in range(0, 2000):

    ombh2 = params[i, 0]
    omch2 = params[i, 1]
    H0 = params[i, 2]

    # Only plot vals with params in this specific window

    if ombh2 > 0.015 and ombh2 < 0.04 and omch2 > 0.03 and omch2 < 0.2 and H0 > 45 and H0 < 95:
        if counter > 100:
            continue
        
        true = real_lensing[i]
    
        fit, poly, sigm = lensing_tail_fit(ells, ombh2, omch2)
    
        #plt.semilogx(ells, 100*np.abs((fit - true)/true))
        #plt.semilogx(ells, poly*sigm + 1)
        plt.semilogx(ells, 100*np.abs((true - fit)/true))
        
        counter += 1


plt.ylim(0, 100)
plt.xlim(1000, 5000)


NameError: name 'np' is not defined